In [1]:
import sys
sys.path.insert(-1, '/xdisk/msurdeanu/fanluo/miniconda3/lib/python3.7/site-packages')

import torch
from torch.nn import functional as F
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
import os
from torchvision import datasets, transforms
 

In [10]:
class LitMNIST(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.layer_1 = torch.nn.Linear(28 * 28, 128)
        self.layer_2 = torch.nn.Linear(128, 256)
        self.layer_3 = torch.nn.Linear(256, 10)

    def forward(self, x):
        batch_size, channels, width, height = x.size()
        x = x.view(batch_size, -1)
        x = self.layer_1(x)
        x = torch.relu(x)
        x = self.layer_2(x)
        x = torch.relu(x)
        x = self.layer_3(x)
        x = torch.log_softmax(x, dim=1)
        return x

    @pl.data_loader
    def train_dataloader(self):
        return DataLoader(MNIST(os.getcwd(), train=True, download=True,
                          transform=transforms.ToTensor()), batch_size=32)

    @pl.data_loader
    def test_dataloader(self):
        # OPTIONAL
        # can also return a list of test dataloaders
        return DataLoader(MNIST(os.getcwd(), train=False, download=True,
                          transform=transforms.ToTensor()), batch_size=32)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)

        # add logging
        logs = {'loss': loss}
        return {'loss': loss, 'log': logs}

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        return {'val_loss': loss}

    def test_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}
 

In [12]:
model = LitMNIST()
trainer = Trainer(max_epochs=2)
# trainer = Trainer(gpus=1, max_epochs=2)
# trainer = Trainer(gpus=2, max_epochs=2, distributed_backend='dpp')
trainer.fit(model)

INFO:root:gpu available: True, used: True
INFO:root:VISIBLE GPUS: 0
INFO:root:
      Name    Type Params
0  layer_1  Linear  100 K
1  layer_2  Linear   33 K
2  layer_3  Linear    2 K


Epoch 1: 100%|██████████| 1875/1875 [00:12<00:00, 145.47batch/s, batch_idx=1874, gpu=0, loss=0.101, v_num=5]


/xdisk/msurdeanu/fanluo/miniconda3/envs/hotpotqa/lib/python3.6/site-packages/pytorch_lightning/callbacks/pt_callbacks.py:144: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  RuntimeWarning)


1

In [11]:
# run test set
trainer.test()

MisconfigurationException: You called `.test()` without defining model's `.test_step()` or `.test_end()`.
                    Please define and try again

Ref: <br>
[1] https://pytorch-lightning.readthedocs.io/en/0.7.5/introduction_guide.html <br>
[2] https://pytorch-lightning.readthedocs.io/en/0.6.0/lightning-module.html?highlight=MNIST